## Reminder
Please use Python 3.10 or Python 3.9 to run the code. Otherwise the pd.json_normalize function may not work

## Data collection idea
* Use "Board" dataset as a bridge to link both "Sprint" and "Issue" URL and data table.
* For spint and issues table, attach create a new column and store board id in it, so that we can know which board does the data comes from.

### Steps: 
#### Board: 
1. Retrieve board from API URL: {Base URL of your project}/rest/agile/1.0/board?projectKeyOrId={projectKey}
* Base URL: check the excel file on OneDrive
* ProjectKey: Search the project name on corresponding platform (RedHat or Apache) -> Under the "Project" category
2. Request data by using GET method and get the json output.
3. Transform data from JSON format to the pandas dataframe.
4. Filter the board type (type == 'scrum') and reset the index
5. Store board_id for future analysis
6. Save as csv file so that we don't have to rerun the code every time.

#### Sprint: 
1. Loop pre-stored board_id to get all sprint and store them into an empty list (as computation cost of expand a cost is much smaller than expand a pandas dataframe).
2. Normalize the list and we will get a dataframe (called df_initalResult) whose records are all nested.
3. For each row, we want to flatten the data into a normal data record type. Therefore, we create a new empty dataframe (df_finalResult) first then loop through the "df_initalResult" by using its index and save each records in a temp dataframe (df_temp). Then concat df_finalResult and df_temp, after which, we can get a complete datasets that contains all records.
4. Save as csv file.

#### Issues:
1. Similar as Sprint method. The thing is that we only want the issues that have story points, which is stored in customer fields (check column name in excel file). Therefore, after we getting all issues, we need to filter out the issues that have story points.
2. Save as csv file.

In [1]:
import requests
from requests.auth import HTTPBasicAuth
import json
import base64
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

## Your Project Name

In [2]:
#For RedHat, just replace your token and keep string "Bearer"
auth = "Bearer YourToken"

#For Apache, you can use same method as RedHat, if it doesn't work, please try the following
auth=HTTPBasicAuth("YourUserName", "YourPassword")
# Also for Apache, please use following format to retrieve data.
url = "xxxxxx"
headers = {
   "Accept": "application/json"
}

payload = {
   "maxResults":50 
}

board = requests.request("GET",url,headers=headers,auth=auth, params=payload).json()

### Board

In [6]:
# Replace BaseURL and project key with your own project
board_url = '{BaseURL}/rest/agile/1.0/board?projectKeyOrId={ProjectKey}'

headers = {
   "Content-Type": "application/json",
   "Authorization": auth,  
}

payload = {
   "maxResults":50 #100, 20
}

# Request data by using GET method and directly get the json output
board = requests.request("GET",board_url,headers=headers, params=payload).json()

# As we only intreseted in the "values" in the board, so we only normalize the board['values']
board_df = pd.json_normalize(board['values'])

# Filter the scrum board type and maintain the pd dataframe formate
board_df = board_df[board_df['type']=='scrum']

# Reset the index for the future convience.
board_df = board_df.reset_index()

# Store the board id into variable board_id
board_id = board_df['id']

#Save the df as csv file. Use your project name to define the file name
board_df.to_csv('YourProjectName_board.csv') 

#### Sprint

In [7]:
headers = {
   "Content-Type": "application/json",
   "Authorization": auth,  
}


payload_sprint = {
'maxResults':2000  #Retrieve at most 2000 records, can be modified based on your own project
}

list1 = [] 

# Loop through the required board, and store the records into the empty list1
for i in board_id:
   sprint_url = 'https://issues.redhat.com/rest/agile/1.0/board/' + f'{i}' + '/sprint'
   sprint = requests.request("GET",sprint_url ,headers=headers, params=payload_sprint).json()
   try:
      list1.append(sprint['values'])
   except:
      pass

In [7]:
# Normalize the list and get the inital dataframe, whose record are nested.
sprint_df = pd.json_normalize(list1)
sprint_df.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,"{'id': 5140, 'self': 'https://issues.redhat.co...","{'id': 5408, 'self': 'https://issues.redhat.co...","{'id': 5504, 'self': 'https://issues.redhat.co...","{'id': 6024, 'self': 'https://issues.redhat.co...","{'id': 6126, 'self': 'https://issues.redhat.co...","{'id': 6146, 'self': 'https://issues.redhat.co...","{'id': 6180, 'self': 'https://issues.redhat.co...","{'id': 6257, 'self': 'https://issues.redhat.co...","{'id': 6287, 'self': 'https://issues.redhat.co...","{'id': 6363, 'self': 'https://issues.redhat.co...",...,"{'id': 7764, 'self': 'https://issues.redhat.co...","{'id': 7912, 'self': 'https://issues.redhat.co...","{'id': 7914, 'self': 'https://issues.redhat.co...","{'id': 7916, 'self': 'https://issues.redhat.co...","{'id': 7918, 'self': 'https://issues.redhat.co...","{'id': 8160, 'self': 'https://issues.redhat.co...","{'id': 8158, 'self': 'https://issues.redhat.co...","{'id': 8162, 'self': 'https://issues.redhat.co...","{'id': 8164, 'self': 'https://issues.redhat.co...","{'id': 8166, 'self': 'https://issues.redhat.co..."
1,"{'id': 3262, 'self': 'https://issues.redhat.co...","{'id': 3342, 'self': 'https://issues.redhat.co...","{'id': 3352, 'self': 'https://issues.redhat.co...","{'id': 3362, 'self': 'https://issues.redhat.co...","{'id': 3372, 'self': 'https://issues.redhat.co...","{'id': 3382, 'self': 'https://issues.redhat.co...","{'id': 3392, 'self': 'https://issues.redhat.co...","{'id': 3962, 'self': 'https://issues.redhat.co...","{'id': 4462, 'self': 'https://issues.redhat.co...","{'id': 4520, 'self': 'https://issues.redhat.co...",...,"{'id': 38868, 'self': 'https://issues.redhat.c...","{'id': 38869, 'self': 'https://issues.redhat.c...",None,None,None,None,None,None,None,None
2,"{'id': 3222, 'self': 'https://issues.redhat.co...","{'id': 3352, 'self': 'https://issues.redhat.co...","{'id': 3362, 'self': 'https://issues.redhat.co...","{'id': 3372, 'self': 'https://issues.redhat.co...","{'id': 3382, 'self': 'https://issues.redhat.co...","{'id': 3392, 'self': 'https://issues.redhat.co...","{'id': 4848, 'self': 'https://issues.redhat.co...","{'id': 4660, 'self': 'https://issues.redhat.co...","{'id': 4412, 'self': 'https://issues.redhat.co...","{'id': 4896, 'self': 'https://issues.redhat.co...",...,"{'id': 6287, 'self': 'https://issues.redhat.co...","{'id': 6363, 'self': 'https://issues.redhat.co...","{'id': 6365, 'self': 'https://issues.redhat.co...","{'id': 6377, 'self': 'https://issues.redhat.co...","{'id': 6387, 'self': 'https://issues.redhat.co...","{'id': 6627, 'self': 'https://issues.redhat.co...","{'id': 6633, 'self': 'https://issues.redhat.co...","{'id': 6531, 'self': 'https://issues.redhat.co...","{'id': 6851, 'self': 'https://issues.redhat.co...","{'id': 6725, 'self': 'https://issues.redhat.co..."
3,"{'id': 8166, 'self': 'https://issues.redhat.co...","{'id': 8514, 'self': 'https://issues.redhat.co...","{'id': 8516, 'self': 'https://issues.redhat.co...","{'id': 8518, 'self': 'https://issues.redhat.co...","{'id': 8520, 'self': 'https://issues.redhat.co...","{'id': 8932, 'self': 'https://issues.redhat.co...","{'id': 8934, 'self': 'https://issues.redhat.co...","{'id': 8936, 'self': 'https://issues.redhat.co...","{'id': 8938, 'self': 'https://issues.redhat.co...","{'id': 9122, 'self': 'https://issues.redhat.co...",...,"{'id': 25171, 'self': 'https://issues.redhat.c...","{'id': 25172, 'self': 'https://issues.redhat.c...","{'id': 25173, 'self': 'https://issues.redhat.c...","{'id': 25174, 'self': 'https://issues.redhat.c...","{'id': 25175, 'self': 'https://issues.redhat.c...","{'id': 38867, 'self': 'https://issues.redhat.c...","{'id': 38868, 'self': 'https://issues.redhat.c...","{'id': 38869, 'self': 'https://issues.redhat.c...","{'id': 40100, 'self': 'https://issues.redhat.c...","{'id': 40625, 'self': 'https://issues.redhat.c..."
4,"{'id': 4660, 'self': 'https://issues.redhat.co...","{'id': 5140, 'self': 'https://issues.redhat.co...","{'id': 5214, 'self': 'https://

In [20]:
#Create an empty dataframe
sprint_result = pd.DataFrame()
sprint_result.head()

# sprint_df.index is the row number. For each row, we need to flatten the records and store them into a dataframe. Our goal is to flatten all these record and generate a single dataframe.
for i in sprint_df.index:
   # Flatten records in a row based on row index
   sprint_df_1 = sprint_df.iloc[i].apply(pd.Series) 
   # Attach corresponding board_id
   sprint_df_1['board_id'] = board_df['id'][i]
   # Create a list called "frame" to tell python we want to contact two dataframs together -- sprint_result and sprint_df_1
   frame = [sprint_result, sprint_df_1]
   #Contact two dataframes
   sprint_result = pd.concat(frame)

In [23]:
# Filter the sprint that totally empty
sprint_result = sprint_result[sprint_result['id'].notna()]
sprint_result.head()

,id,self,state,name,startDate,endDate,completeDate,activatedDate,originBoardId,goal,board_id
0,5140.0,https://issues.redhat.com/rest/agile/1.0/sprin...,closed,GSSP_6.4 Sprint- 3-31 Aug,2016-08-03T08:17:20.067Z,2016-08-31T12:17:00.000Z,2016-09-06T21:40:41.465Z,2016-08-03T08:17:20.067Z,3548.0,NaN,4532
1,5408.0,https://issues.redhat.com/rest/agile/1.0/sprin...,closed,November Sprint,2016-11-03T23:46:23.504Z,2016-12-02T00:46:00.000Z,2016-11-30T13:29:23.663Z,2016-11-03T23:46:23.504Z,3548.0,NaN,4532
2,5504.0,https://issues.redhat.com/rest/agile/1.0/sprin...,closed,December Sprint,2016-12-01T13:10:38.253Z,2016-12-29T13:10:00.000Z,2017-01-02T09:17:34.512Z,2016-12-01T13:10:38.253Z,3548.0,NaN,4532
3,6024.0,https://issues.redhat.com/rest/agile/1.0/sprin...,closed,May 2017 Sprint,2017-05-04T20:52:59.038Z,2017-05-29T20:52:00.000Z,2017-06-07T11:15:40.285Z,2017-05-04T20:52:59.038Z,3548.0,NaN,4532
4,6126.0,https://issues.redhat.com/rest/agile/1.0/sprin...,closed,2017 Week 22-23,2017-05-29T04:00:00.000Z,2017-06-12T03:59:00.000Z,2017-06-12T13:25:18.855Z,2017-05-29T04:00:00.000Z,3838.0,NaN,4532


In [24]:
# Reset the index
sprint_result = sprint_result.reset_index()
# Drop the origianl index column
sprint_result = sprint_result.drop(columns=['index'])
sprint_result.head()

,id,self,state,name,startDate,endDate,completeDate,activatedDate,originBoardId,goal,board_id
0,5140.0,https://issues.redhat.com/rest/agile/1.0/sprin...,closed,GSSP_6.4 Sprint- 3-31 Aug,2016-08-03T08:17:20.067Z,2016-08-31T12:17:00.000Z,2016-09-06T21:40:41.465Z,2016-08-03T08:17:20.067Z,3548.0,NaN,4532
1,5408.0,https://issues.redhat.com/rest/agile/1.0/sprin...,closed,November Sprint,2016-11-03T23:46:23.504Z,2016-12-02T00:46:00.000Z,2016-11-30T13:29:23.663Z,2016-11-03T23:46:23.504Z,3548.0,NaN,4532
2,5504.0,https://issues.redhat.com/rest/agile/1.0/sprin...,closed,December Sprint,2016-12-01T13:10:38.253Z,2016-12-29T13:10:00.000Z,2017-01-02T09:17:34.512Z,2016-12-01T13:10:38.253Z,3548.0,NaN,4532
3,6024.0,https://issues.redhat.com/rest/agile/1.0/sprin...,closed,May 2017 Sprint,2017-05-04T20:52:59.038Z,2017-05-29T20:52:00.000Z,2017-06-07T11:15:40.285Z,2017-05-04T20:52:59.038Z,3548.0,NaN,4532
4,6126.0,https://issues.redhat.com/rest/agile/1.0/sprin...,closed,2017 Week 22-23,2017-05-29T04:00:00.000Z,2017-06-12T03:59:00.000Z,2017-06-12T13:25:18.855Z,2017-05-29T04:00:00.000Z,3838.0,NaN,4532


In [25]:
# Save as csv file
sprint_result.to_csv('YourProjectName_sprint.csv')

#### Issues


In [26]:
headers = {
   "Content-Type": "application/json",
   "Authorization": auth,  
}

payload_issues = {
'maxResults':3000
}

list1 = []
# Loop through the required board, and store the records into the empty list1
for i in board_id:
   issues_url = 'https://issues.redhat.com/rest/agile/1.0/board/' + f'{i}' + '/issue'
   issues = requests.request("GET",issues_url ,headers=headers, params=payload_issues).json()
   try:
      list1.append(issues['issues'])
   except:
      pass

In [28]:
# Normalize the list and get the inital dataframe, whose record are nested.
issue_df = pd.json_normalize(list1)
issue_df.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,"{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...",...,"{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation..."
1,"{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...",...,None,None,None,None,None,None,None,None,None,None
2,"{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...",...,"{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation..."
3,"{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...",...,"{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation..."
4,"{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand': 'operations,versionedRepresentation...","{'expand':

In [30]:
#Check a single record content
issue_df.iloc[0][0]

{'expand': 'operations,versionedRepresentations,editmeta,changelog,renderedFields',
 'id': '13429632',
 'self': 'https://issues.redhat.com/rest/agile/1.0/issue/13429632',
 'key': 'BXMSDOC-8068',
 'fields.issuetype.self': 'https://issues.redhat.com/rest/api/2/issuetype/3',
 'fields.issuetype.id': '3',
 'fields.issuetype.description': 'A task that needs to be done.',
 'fields.issuetype.iconUrl': 'https://issues.redhat.com/secure/viewavatar?size=xsmall&avatarId=13278&avatarType=issuetype',
 'fields.issuetype.name': 'Task',
 'fields.issuetype.subtask': False,
 'fields.issuetype.avatarId': 13278,
 'fields.customfield_12318341': None,
 'fields.timespent': None,
 'fields.customfield_12320940': None,
 'fields.project.self': 'https://issues.redhat.com/rest/api/2/project/12316327',
 'fields.project.id': '12316327',
 'fields.project.key': 'BXMSDOC',
 'fields.project.name': 'BxMS Documentation',
 'fields.project.projectTypeKey': 'software',
 'fields.project.avatarUrls.48x48': 'https://issues.redha

In [31]:
issue_result = pd.DataFrame()

for i in issue_df.index:
   issue_df_1 = issue_df.iloc[i].apply(pd.Series)
   issue_df_1['board_id'] = board_df['id'][i]
   frame = [issue_result, issue_df_1]
   issue_result = pd.concat(frame)

issue_result = issue_result.reset_index()
issue_result = issue_result.drop(columns=['index'])

In [40]:
#Only select issues that have story points
issue_result = issue_result[issue_result['fields.customfield_12310243'].notna()]
issue_result.head()

,expand,id,self,key,fields.issuetype.self,fields.issuetype.id,fields.issuetype.description,fields.issuetype.iconUrl,fields.issuetype.name,fields.issuetype.subtask,...,fields.customfield_12316243.avatarUrls.16x16,fields.customfield_12316243.avatarUrls.32x32,fields.customfield_12316243.displayName,fields.customfield_12316243.active,fields.customfield_12316243.timeZone,fields.customfield_12310022,fields.customfield_12310021,fields.customfield_12310071,fields.customfield_12316042,fields.customfield_12316043
0,"operations,versionedRepresentations,editmeta,c...",13429632,https://issues.redhat.com/rest/agile/1.0/issue...,BXMSDOC-8068,https://issues.redhat.com/rest/api/2/issuetype/3,3,A task that needs to be done.,https://issues.redhat.com/secure/viewavatar?si...,Task,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56,"operations,versionedRepresentations,editmeta,c...",12835934,https://issues.redhat.com/rest/agile/1.0/issue...,BXMSDOC-4879,https://issues.redhat.com/rest/api/2/issuetype/3,3,A task that needs to be done.,https://issues.redhat.com/secure/viewavatar?si...,Task,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,"operations,versionedRepresentations,editmeta,c...",12762960,https://issues.redhat.com/rest/agile/1.0/issue...,AF-1456,https://issues.redhat.com/rest/api/2/issuetype/3,3,A task that needs to be done.,https://issues.redhat.com/secure/viewavatar?si...,Task,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,"operations,versionedRepresentations,editmeta,c...",12768016,https://issues.redhat.com/rest/agile/1.0/issue...,AF-1542,https://issues.redhat.com/rest/api/2/issuetype/3,3,A task that needs to be done.,https://issues.redhat.com/secure/viewavatar?si...,Task,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,"operations,versionedRepresentations,editmeta,c...",12766916,https://issues.redhat.com/rest/agile/1.0/issue...,AF-1520,https://issues.redhat.com/rest/api/2/issuetype/1,1,A problem which impairs or prevents the functi...,https://issues.redhat.com/secure/viewavatar?si...,Bug,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
#Check the dataframe shape
issue_result.shape

(2610, 336)

In [42]:
#Save as csv file
issue_result.to_csv('YourProjectName_issues.csv')